In [ ]:
from nltk.tokenize import RegexpTokenizer
import stop_words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim import corpora, models
import gensim
import operator

from gensim.models import Phrases
from collections import defaultdict

import string

import pickle as pk
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

In [ ]:
sns.set()

In [ ]:
addressBook = {"serializedData" : "D:/PhD/Projects/Defected Info Bibliometrics Study/Special Issue Effort/Analysis/ProcessedData/SerializedData/"}

In [ ]:
allPapers = pk.loads(open(addressBook["serializedData"]+"allPapers.dic", "rb").read())
annualPapers = pk.loads(open(addressBook["serializedData"]+"annualPapers.dic", "rb").read())

MAKING THE STOPWORDS LIST

In [ ]:
stopWords1 = set(stopwords.words('english'))
stopWords2 = stop_words.get_stop_words('en')
for a in string.ascii_letters:
    stopWords2.append(a)
for i in range(0,400):
    stopWords2.append(str(i))
for i in ["One","Two","Three","Four","Five","Six","Seven","Eight","Nine","Ten","one","two","three","four","five","six","seven","eight","nine","ten","also","since","paper","article","result", "study"]:
    stopWords2.append(i)
stopwords = list(set(list(stopWords1) + stopWords2))

LEMMATIZATION

In [ ]:
lemmatizer = WordNetLemmatizer()

PHRASE DETECTION

In [ ]:
documents = []
sentence_stream = []
for pid in tqdm(allPapers):
    ptitle = allPapers[pid]["TI"]
    pabstract = allPapers[pid]["AB"]
    pauthorKeywords = allPapers[pid]["DE"].replace(";", " ")
    pplusKeywords = allPapers[pid]["ID"].replace(";", " ")
    pcontent = pauthorKeywords + pplusKeywords + ptitle + pabstract
    documents.append(pcontent)
sentence_stream.extend([doc.split(" ") for doc in documents])
bigram = Phrases(sentence_stream, min_count=1, threshold=2)

In [ ]:
for topicNum in range(1,6):
    LDAs = {}
    wordNum = 5
    passNum = 50
    texts = {"1900~1980":[],"1980~1990":[],"1990~2000":[],"2000~2010":[],"2010~2018":[]}
    adjustedTexts = {"1900~1980": [], "1980~1990": [], "1990~2000": [], "2000~2010": [], "2010~2018": []}
    wholetexts = []

    for paper_id in allPapers:
            title = allPapers[paper_id]["TI"]
            abstract = allPapers[paper_id]["AB"]
            authorKeywords = allPapers[paper_id]["DE"].replace(";", " ")
            plusKeywords = allPapers[paper_id]["ID"].replace(";", " ")
            content = (authorKeywords + plusKeywords + title + abstract).lower()
            content = content.replace("rumor","").replace("rumour","").replace("fallacy","").replace("fallacies","").replace("misinformation","").replace("gossip","").replace("pseudoscience","").replace("smear campaigns","").replace("smear campaign","").replace("factchecking","").replace("fact checking",""). replace("fact-checking","").replace("fact-check","").replace("fact check","").replace("factcheck","").replace("truthiness","").replace("media manipulation","").replace("urban-legend","").replace("urban legend","").replace("alternative facts","").replace("alternative fact","").replace("post truth", "").replace("kompromat","").replace("false flag","").replace("denial and deception", "").replace("propaganda","").replace("fallacy","").replace("factoid","").replace("disinformation","").replace("defamation","").replace("character assassination","").replace("common misconceptions","").replace("common misconception","").replace("false dilemma","").replace("fake news","").replace("fake-news","").replace("hoax","").replace("blind items","").replace("blind item","")       


            #Preprocessing
            tokens = list(gensim.utils.tokenize(content, deacc=True, lower=True))
            lemmatizedTokens = [lemmatizer.lemmatize(lw) for lw in tokens if lw not in stopwords]
            phrases = bigram[lemmatizedTokens]
            lemmatizedTokens = list(set(lemmatizedTokens + phrases))

            year = int(allPapers[paper_id]["PY"].strip())
            if(year in [y for y in range(1900,1980)]):
                if len(lemmatizedTokens)>0:
                    texts["1900~1980"].append(lemmatizedTokens)
            elif(year in [y for y in range(1980,1990)]):
                if len(lemmatizedTokens)>0:
                    texts["1980~1990"].append(lemmatizedTokens)
            elif(year in [y for y in range(1990,2000)]):
                if len(lemmatizedTokens)>0:
                    texts["1990~2000"].append(lemmatizedTokens)
            elif(year in [y for y in range(2000,2010)]):
                if len(lemmatizedTokens)>0:
                    texts["2000~2010"].append(lemmatizedTokens)
            elif(year in [y for y in range(2010,2019)]):
                if len(lemmatizedTokens)>0:
                    texts["2010~2018"].append(lemmatizedTokens)
            if len(lemmatizedTokens)>0:
                wholetexts.append(lemmatizedTokens)

    frequency = defaultdict(int)
    for text in wholetexts:
        for token in text:
            frequency[token] += 1
    adjustedWholeText = [[token for token in text if frequency[token] > 1] for text in wholetexts]
    freq = sorted_x = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)

    for key in texts:
        adjustedTexts[key] = [[token for token in text if frequency[token] > 1] for text in texts[key]]

    dictionary = corpora.Dictionary(adjustedWholeText)
    corpus = [dictionary.doc2bow(text) for text in adjustedWholeText]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topicNum, id2word=dictionary, passes=passNum)
    LDAs["total"] = ldamodel
    print(ldamodel.print_topics(num_topics=topicNum, num_words=wordNum))
    LDA_temp = {"total": [], "1900~1980":[],"1980~1990":[],"1990~2000":[],"2000~2010":[],"2010~2018":[]}
    df = pd.DataFrame(columns=["word"+str(i) for i in range(1,topicNum*wordNum+1)])

    for c,eachYear in enumerate(adjustedTexts):
        if len(adjustedTexts[eachYear])>0:
            dictionary = corpora.Dictionary(adjustedTexts[eachYear])
            corpus = [dictionary.doc2bow(text) for text in adjustedTexts[eachYear]]
            ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topicNum, id2word=dictionary, passes=passNum)
            LDAs[eachYear] = ldamodel
            print(str(eachYear) + " : ")
            print(ldamodel.print_topics(num_topics=topicNum, num_words=wordNum))

    open(f'./LDA(firstVersion)_{str(passNum)}_{str(topicNum)}_{str(wordNum)}.dic', "wb").write(pk.dumps(LDAs))

    for c,eachYear in enumerate(LDAs):
        for i in range(0,topicNum):
            LDA_temp[eachYear].extend([item.split("\"")[1].strip().rstrip() for item in str(LDAs[eachYear].print_topics(num_topics=topicNum, num_words=wordNum)[i]).split("+")])
            while(len(LDA_temp[eachYear]) % wordNum != 0):
                LDA_temp[eachYear].append(0)
        df.loc[c] = [i for i in LDA_temp[eachYear]]
    df.to_csv(f'./LDA_(firstVersion)_(Keywords+Title+Abstract){str(topicNum)}.csv', sep=';', decimal=',')


In [ ]:
for topicNum in tqdm(range(1,6)):
    LDAs = {}
    wordNum = 5
    passNum = 50
    texts = {"1900~1970":[],"1970~2018":[]}
    adjustedTexts = {"1900~1970": [], "1970~2018": []}
    wholetexts = []

    for paper_id in tqdm(allPapers):
            title = allPapers[paper_id]["TI"]
            abstract = allPapers[paper_id]["AB"]
            authorKeywords = allPapers[paper_id]["DE"].replace(";", " ")
            plusKeywords = allPapers[paper_id]["ID"].replace(";", " ")
            content = (authorKeywords + plusKeywords + title + abstract).lower().replace("rumor","rumour").replace("rumour","").replace("fallacy","").replace("misinformation","").replace("gossip","").replace("pseudoscience","").replace("smear campaigns","").replace("smear campaign","").replace("factchecking","").replace("fact checking",""). replace("fact-checking","").replace("fact-check","").replace("fact check","").replace("factcheck","").replace("truthiness","").replace("media manipulation","").replace("urban-legend","").replace("urban legend","").replace("alternative facts","").replace("alternative fact","").replace("post truth", "").replace("kompromat","").replace("false flag","").replace("denial and deception", "").replace("propaganda","").replace("fallacy","").replace("factoid","").replace("disinformation","").replace("defamation","").replace("character assassination","").replace("common misconceptions","").replace("common misconception","").replace("false dilemma","").replace("fake news","").replace("fake-news","").replace("hoax","").replace("blind items","").replace("blind item","")       

            #Preprocessing
            tokens = list(gensim.utils.tokenize(content, deacc=True, lower=True))
            lemmatizedTokens = [lemmatizer.lemmatize(lw) for lw in tokens if lw not in stopwords]
            phrases = bigram[lemmatizedTokens]
            lemmatizedTokens = list(set(lemmatizedTokens + phrases))

            year = int(allPapers[paper_id]["PY"].strip())
            if(year in [y for y in range(1900,1970)]):
                if len(lemmatizedTokens)>0:
                    texts["1900~1970"].append(lemmatizedTokens)
            elif(year in [y for y in range(1970,2019)]):
                if len(lemmatizedTokens)>0:
                    texts["1970~2018"].append(lemmatizedTokens)
            if len(lemmatizedTokens)>0:
                wholetexts.append(lemmatizedTokens)

    frequency = defaultdict(int)
    for text in wholetexts:
        for token in text:
            frequency[token] += 1
    adjustedWholeText = [[token for token in text if frequency[token] > 1] for text in wholetexts]
    freq = sorted_x = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)

    for key in texts:
        adjustedTexts[key] = [[token for token in text if frequency[token] > 1] for text in texts[key]]

    dictionary = corpora.Dictionary(adjustedWholeText)
    corpus = [dictionary.doc2bow(text) for text in adjustedWholeText]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topicNum, id2word=dictionary, passes=passNum)
    LDAs["total"] = ldamodel
    print(ldamodel.print_topics(num_topics=topicNum, num_words=wordNum))
    LDA_temp = {"total": [], "1900~1970":[],"1970~2018":[]}
    df = pd.DataFrame(columns=["word"+str(i) for i in range(1,topicNum*wordNum+1)])

    for c,eachYear in enumerate(adjustedTexts):
        if len(adjustedTexts[eachYear])>0:
            dictionary = corpora.Dictionary(adjustedTexts[eachYear])
            corpus = [dictionary.doc2bow(text) for text in adjustedTexts[eachYear]]
            ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topicNum, id2word=dictionary, passes=passNum)
            LDAs[eachYear] = ldamodel
            print(str(eachYear) + " : ")
            print(ldamodel.print_topics(num_topics=topicNum, num_words=wordNum))

    open(f'./LDA(SecondVersion)_{str(passNum)}_{str(topicNum)}_{str(wordNum)}.dic', "wb").write(pk.dumps(LDAs))

    for c,eachYear in enumerate(LDAs):
        for i in range(0,topicNum):
            LDA_temp[eachYear].extend([item.split("\"")[1].strip().rstrip() for item in str(LDAs[eachYear].print_topics(num_topics=topicNum, num_words=wordNum)[i]).split("+")])
            while(len(LDA_temp[eachYear]) % wordNum != 0):
                LDA_temp[eachYear].append(0)
        df.loc[c] = [i for i in LDA_temp[eachYear]]
    df.to_csv(f'./LDA(FirstVersion)_(Keywords+Title+Abstract)_{str(topicNum)}.csv', sep=';', decimal=',')

Loading the serialized LDA

In [ ]:
import os

In [ ]:
next(os.walk("."))
LDA = pk.loads(open("./LDA(firstVersion)_50_5_5.dic","rb").read())

In [ ]:
lda1 = LDA["1900~1980"]
lda2 = LDA["1980~1990"]
lda3 = LDA["1990~2000"]
lda4= LDA["2000~2010"]
lda5= LDA["2010~2018"]    

In [ ]:
word2id = {}
topicNum = 5
id_counter = 1
for year, lda in LDA.items():
    for tn in tqdm(range(topicNum)):
        elements = lda.show_topic(tn, topn=lda.num_terms)
        for element in elements:
            if element[0] not in word2id.keys():
                word2id[element[0]] = id_counter
                id_counter+=1     

In [ ]:
vectors = {}
topicNum = 5
for year, lda in LDA.items():
    vectors[year] = []
    for tn in tqdm(range(topicNum)):
        elements = lda.show_topic(tn, topn=lda.num_terms)
        vectors[year].append([(word2id[element[0]], element[1]) for element in elements])

In [ ]:
from gensim import matutils

In [ ]:
matutils.cossim(vectors["total"][1], vectors["1900~1980"][4])

In [ ]:
del vectors["total"]

In [ ]:
columns = []
for i in vectors:
    for j in range(5):
        columns.append(f'{i} (topic{j})')
df = pd.DataFrame(columns=columns)
for j in range(25):
    df.loc[j] = [0 for k in range(25)]
df.index = columns

In [ ]:
vec1 = vectors.copy()
vec2 = vectors.copy()

In [ ]:
topic_num = 5
for ind1, year1 in enumerate(vec1): #row - first level
    for ind2, year2 in enumerate(vec2): #column - first level
            for ind11, tn1 in enumerate(range(topic_num)): #row - second level
                for ind22, tn2 in enumerate(range(topic_num)): #column - second level
                        sim = matutils.cossim(vec1[year1][tn1], vec2[year2][tn2])
                        df.loc[f'{year1} (topic{tn1})'][f'{year2} (topic{tn2})'] = sim

In [ ]:
df.to_csv("LDA_MatrixForUncertainty.csv", sep=";", decimal=",", float_format=".3f" )

Creating a chart to display dynamic of change in novelty level with two approaches

In [ ]:
df = pd.read_csv("LDA_MatrixForUncertainty.csv", sep=";", decimal=",")

In [ ]:
df2 = pd.DataFrame(columns=["Period", "Maximum Novelty", "Average Novelty"])

In [ ]:
df2.loc[0]=["P1", 0.57, 0.68]
df2.loc[1]=["P2", 0.66, 0.70]
df2.loc[2]=["P3", 0.27, 0.48]
df2.loc[3]=["P4", 0.13, 0.41]

In [ ]:
df2.plot()
plt.ylabel("Novelty level")
plt.xlabel("Periods")
plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False)
plt.savefig("_sns_noveltyLevel.pdf")
plt.show()

In [ ]:
df3 = pd.DataFrame(columns=["period", "uncertainty"])
df3.loc[0]=["P0", 0.57]
df3.loc[1]=["P1", 0.63]
df3.loc[2]=["P2", 0.41]
df3.loc[3]=["P3", 0.12]

In [ ]:
df3.plot(legend=False)
plt.ylabel("uncertainty level")
plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False)
plt.show()